<a href="https://colab.research.google.com/github/HenryZumaeta/ReprojectCoordinates/blob/main/ReprojectCoordinates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **<font color='F1C40F'> Reproject Coodinates - Shared by [Henry Zumaeta](https://github.com/HenryZumaeta) </font>**


In [ ]:
# Subir archivo CSV
from google.colab import files
import pandas as pd
import geopandas as gpd
from pyproj import CRS, Transformer

# Permitir la subida de archivos
uploaded = files.upload()

# Leer el archivo CSV subido
for filename in uploaded.keys():
    df = pd.read_csv(filename)
    print(f"Archivo {filename} subido y cargado en un DataFrame")
    print(df.head())


In [ ]:
# @markdown <br><center><img src='https://raw.githubusercontent.com/HenryZumaeta/DataProgrammingRepo/main/db/UMT%20a%20Geogr%C3%A1ficas.png' height="50" alt="UTM2Geo-logo"/></center>
# @markdown <center><h2>UTM a Geográficas</h2></center><br>

# @markdown <h3>Datos de entrada: Ingresar las propiedades</h3></center><br>

# Definir rutas de origen y destino
INPUT_ZONA = "32T"  #@param {type:"string"}
OUTPUT_DATUM = "WGS84"  #@param {type:"string"}
OUTPUT_FILENAME = "coordenadas_convertidas.csv"  #@param {type:"string"}

print(f"Zona UTM: {INPUT_ZONA}")
print(f"Datum de salida: {OUTPUT_DATUM}")
print(f"Nombre del archivo de salida: {OUTPUT_FILENAME}")

# Función para convertir UTM a Geográficas
def convertir_utm_a_geografica(df, zona, datum):
    # Definir CRS (Coordinate Reference System)
    crs_utm = CRS(f"+proj=utm +zone={zona[:-1]} +datum=WGS84 +units=m +no_defs")
    crs_geo = CRS(f"EPSG:4326")  # WGS84

    # Crear un transformador
    transformer = Transformer.from_crs(crs_utm, crs_geo)

    # Convertir coordenadas
    df['Latitud'], df['Longitud'] = transformer.transform(df['NORTE'].values, df['ESTE'].values)

    return df

# Convertir coordenadas del DataFrame
df_convertido = convertir_utm_a_geografica(df, INPUT_ZONA, OUTPUT_DATUM)
print("Coordenadas convertidas")

# Guardar el DataFrame convertido en un archivo CSV
df_convertido.to_csv(OUTPUT_FILENAME, index=False)
print(f"Archivo guardado como {OUTPUT_FILENAME}")

# Permitir la descarga del archivo
files.download(OUTPUT_FILENAME)
